# 1.3  chABSA-datasetからDataLoaderを作成

- 本ファイルでは、chABSA-datasetのデータを使用して、感情分析（0：ネガティブ、1：ポジティブ）を2値クラス分類するためのDatasetとDataLoaderを作成する。


# 前処理と単語分割の関数

In [3]:
from janome.tokenizer import Tokenizer
import re
import mojimoji
import string
import MeCab

j_t = Tokenizer()


def tokenizer_mecab(text):
    m_t = MeCab.Tagger('-Owakati -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
    text = m_t.parse(text)  # これでスペースで単語が区切られる
    ret = text.strip().split()  # スペース部分で区切ったリストに変換
    return ret




def preprocessing_text(text):

    # 半角・全角の統一
    text = mojimoji.han_to_zen(text) 
    # 改行、半角スペース、全角スペースを削除
    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('　', '', text)
    text = re.sub(' ', '', text)
    # 数字文字の一律「0」化
    text = re.sub(r'[0-9 ０-９]+', '0', text)  # 数字

    # カンマ、ピリオド以外の記号をスペースに置換
    for p in string.punctuation:
        if (p == ".") or (p == ","):
            continue
        else:
            text = text.replace(p, " ")

    return text


# 前処理関数
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)  # 前処理の正規化
    ret = tokenizer_mecab(text)  # Janomeの単語分割

    return ret


# 動作確認

text = "また、当事業年度末の店舗数につきましては、前事業年度において決定しておりました２店舗、大幅な修繕を要する１店舗及び業績不振の６店舗を加えた合計９店舗を閉鎖しました結果、133店舗となり前事業年度末に比べ９店舗減少いたしました"
print("="*50)
print("原文")
print("="*50)
print(text)
print("="*50)
print("処理後")
print("="*50)
print(tokenizer_with_preprocessing(text))


原文
また、当事業年度末の店舗数につきましては、前事業年度において決定しておりました２店舗、大幅な修繕を要する１店舗及び業績不振の６店舗を加えた合計９店舗を閉鎖しました結果、133店舗となり前事業年度末に比べ９店舗減少いたしました
処理後
['また', '、', '当', '事業年度', '末', 'の', '店舗', '数', 'に', 'つき', 'まし', 'て', 'は', '、', '前', '事業年度', 'において', '決定', 'し', 'て', 'おり', 'まし', 'た', '0', '店舗', '、', '大幅', 'な', '修繕', 'を', '要する', '0', '店舗', '及び', '業績不振', 'の', '0', '店舗', 'を', '加え', 'た', '合計', '0', '店舗', 'を', '閉鎖', 'し', 'まし', 'た', '結果', '、', '0', '店舗', 'と', 'なり', '前', '事業年度', '末', 'に', '比べ', '0', '店舗', '減少', 'いたし', 'まし', 'た']


# DataLoaderの作成

In [4]:
import torchtext

max_length = 256
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, use_vocab=True,
                            lower=True, include_lengths=True, batch_first=True, fix_length=max_length, init_token="<cls>", eos_token="<eos>")
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)
# init_token：全部の文章で、文頭に入れておく単語
# eos_token：全部の文章で、文末に入れておく単語

train_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='train.tsv',
    validation='test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

In [5]:
print('訓練データの数', len(train_ds))
print('テストデータの数', len(test_ds))
print("="*50)
print('1つ目の訓練データ', vars(train_ds[0]))
print("="*50)
print(vars(train_ds[0])['Text'])
print("ラベル=",vars(train_ds[0])['Label'])
print("="*50)
print('1つ目のテストデータ', vars(test_ds[0]))
print("="*50)
print(vars(test_ds[0])['Text'])
print("ラベル＝",vars(test_ds[0])['Label'])

訓練データの数 1970
テストデータの数 843
1つ目の訓練データ {'Text': ['当社', 'グループ', 'を', '取り巻く', '環境', 'は', '、', '実質賃金', 'が', '伸び悩む', 'なか', '、', '消費者', 'の', '皆様', 'の', '生活防衛', '意識', 'の', '高まり', 'や', '節約志向', 'により', '、', '個人消費', 'は', '本格', '的', 'な', '改善', 'に', 'は', '至ら', 'ず', '、', 'また', '少子高齢化', '、', '人口減少', 'による', '社会構造', 'の', '変化', '、', '雇用', '改善', 'に', '伴う', '労働', 'コスト', 'の', '上昇', '、', '企業間', '競争', 'の', '激化', 'など', '、', '引き続き', '厳しい', '状況', 'と', 'なり', 'まし', 'た'], 'Label': '0'}
['当社', 'グループ', 'を', '取り巻く', '環境', 'は', '、', '実質賃金', 'が', '伸び悩む', 'なか', '、', '消費者', 'の', '皆様', 'の', '生活防衛', '意識', 'の', '高まり', 'や', '節約志向', 'により', '、', '個人消費', 'は', '本格', '的', 'な', '改善', 'に', 'は', '至ら', 'ず', '、', 'また', '少子高齢化', '、', '人口減少', 'による', '社会構造', 'の', '変化', '、', '雇用', '改善', 'に', '伴う', '労働', 'コスト', 'の', '上昇', '、', '企業間', '競争', 'の', '激化', 'など', '、', '引き続き', '厳しい', '状況', 'と', 'なり', 'まし', 'た']
ラベル= 0
1つ目のテストデータ {'Text': ['その', '一方', 'で', '、', '中国経済', 'の', '景気減速', 'や', '米国', '新政権', 'の', '政策', '運営', '、', '英国', 'の', 'ｅｕ', '離脱', '等', '

# fastTextの学習済みモデルをダウンロード

fastTextの日本語学習済みモデルを以下からダウンロード


https://qiita.com/Hironsan/items/8f7d35f0a36e0f99752c

のfastTextの「URL2：Download Word Vectors(NEologd)」部分のリンク

https://drive.google.com/open?id=0ByFQ96A4DgSPUm9wVWRLdm5qbmc

からGoogle Driveのリンクに飛んで「vector_neologd.zip」をダウンロードして解凍したfastTextの日本語学習済みモデル「model.vec」を利用する。

# ボキャブラリーを作成

In [7]:
# torchtextで単語ベクトルとして英語学習済みモデルを読み込みます

from torchtext.vocab import Vectors

japanese_fastText_vectors = Vectors(name='./data/model.vec')


print("1単語を表現する次元数：", japanese_fastText_vectors.dim)
print("単語数：", len(japanese_fastText_vectors.itos))

1単語を表現する次元数： 300
単語数： 351122


In [8]:
# ベクトル化したバージョンのボキャブラリーを作成
TEXT.build_vocab(train_ds, vectors=japanese_fastText_vectors, min_freq=5)

# ボキャブラリーのベクトルを確認
print(TEXT.vocab.vectors.shape)  # 1015474個の単語が200次元のベクトルで表現されている
TEXT.vocab.vectors

# ボキャブラリーの単語順番を確認
TEXT.vocab.stoi


torch.Size([996, 300])


defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f584c337ac8>>,
            {'<unk>': 0,
             '<pad>': 1,
             '<cls>': 2,
             '<eos>': 3,
             '0': 4,
             '、': 5,
             'の': 6,
             'は': 7,
             'た': 8,
             'まし': 9,
             '円': 10,
             'に': 11,
             'と': 12,
             '万': 13,
             'が': 14,
             '百': 15,
             'し': 16,
             '％': 17,
             '．': 18,
             '（': 19,
             '）': 20,
             '億': 21,
             'なり': 22,
             'を': 23,
             'で': 24,
             '年度': 25,
             '売上高': 26,
             '連結会計': 27,
             'て': 28,
             '，': 29,
             'により': 30,
             '増': 31,
             '比': 32,
             '減': 33,
             'や': 34,
             '増加': 35,
             '前期比': 36,
             'な': 37,
             '前': 38,
             '前年同期

In [9]:
# DataLoaderを作成
train_dl = torchtext.data.Iterator(train_ds, batch_size=8, train=True)

val_dl = torchtext.data.Iterator(
    test_ds, batch_size=8, train=False, sort=False)


# 動作確認 検証データのデータセットで確認

batch = next(iter(val_dl))
print("="*50)
#単語が数字に変換され、max_lengthに満たない部分はpad=1になっていることが確認できる。
print(batch.Text[0][2])
print("="*50)
print(batch.Label)

tensor([  2,  69,   5, 252,  72,   7,   5, 172,   6,   0,  14, 348,  52, 246,
          5, 112,   7,   0,  30,   0, 186,  23,   0,  11,  16,  28,  78,   5,
        169,  66, 185, 177,   0,  41,   0,  14,   0, 124, 316,  88,  11,  82,
         79,   3,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,  

In [10]:
# '会計': id=45の分散表現の確認
print(TEXT.vocab.vectors[45].shape)
print(TEXT.vocab.vectors[45])

torch.Size([300])
tensor([-7.0466e-02,  4.9462e-02, -2.4484e-01, -4.0976e-02,  1.4057e-01,
         3.8090e-01,  3.8333e-01,  3.3049e-01,  2.9925e-02, -3.5402e-01,
        -5.7561e-03, -1.3193e-01, -5.8086e-01, -3.6471e-01, -1.4834e-01,
         2.9823e-01, -1.3806e-01, -1.4637e-01, -1.1059e-02, -1.4242e-01,
         3.0678e-01,  1.2530e-01,  1.6976e-01, -4.5391e-01,  1.9575e-01,
        -4.4382e-01,  5.1290e-01,  3.1994e-02,  1.7172e-01, -7.9803e-02,
        -2.0055e-02, -3.0086e-01, -1.5612e-02,  9.1992e-02,  2.3463e-01,
        -7.3368e-02,  3.2356e-02,  8.2540e-02, -7.2371e-04, -1.2431e-01,
         1.5609e-01, -4.4805e-02, -3.3459e-01,  6.7891e-02, -4.5369e-02,
        -5.2443e-02, -1.3054e-01, -3.3497e-03, -2.8977e-01,  8.5366e-02,
         1.3593e-01,  1.2522e-01,  9.3992e-02, -2.7340e-01,  2.7430e-01,
        -1.3597e-01,  5.9672e-02, -1.2708e-01,  1.1827e-01,  1.2876e-01,
        -2.7649e-02, -3.3302e-01, -3.7469e-02,  1.0013e-01,  1.1445e-01,
         1.1546e-01,  6.8721e-02,

以上